In [ ]:
import gzip
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

monitor_foldername = "cbrssdr1-browning"
def read_gzipped_csv(file_path):
    with gzip.open(file_path, 'rt') as file:
        df = pd.read_csv(file)
        return df


def extract_timestamp_from_filename(filename):
    # Extract the Unix timestamp using a regular expression
    match = re.search(r'-(\d+)\.csv\.gz$', filename)
    if match:
        return int(match.group(1))
    else:
        return None


def concatenate_csvs_in_folder(folder_path):
    dataframes = []
    cutoff_date = pd.Timestamp('2022-06-01')  # Set the cutoff date to June 1, 2022

    for filename in os.listdir(folder_path):
        if filename.endswith('.gz'):
            file_path = os.path.join(folder_path, filename)
            timestamp = extract_timestamp_from_filename(filename)
            df = read_gzipped_csv(file_path)
            df['timestamp'] = pd.to_datetime(timestamp, unit='s')  # Convert timestamp to datetime

            # Drop rows with NaN values
            df = df.dropna()
            # Filter out rows older than June 2022
            df = df[df['timestamp'] >= cutoff_date]

            dataframes.append(df)

    # Concatenate all dataframes into a single dataframe
    if dataframes:
        return pd.concat(dataframes, ignore_index=True)
    else:
        return pd.DataFrame()


# Example usage
folder_path = './rfbaseline/'+monitor_foldername+"/"
combined_df = concatenate_csvs_in_folder(folder_path)
print(np.unique(combined_df.loc[:30, "timestamp"]))
combined_df = combined_df.drop(columns=['center_freq'])


# Assuming combined_df is your DataFrame


#df = combined_df.groupby('frequency')['power'].apply(avg_pow).reset_index()

# Define the start and end of the CBRS band (in MHz)
band_start = 3470
band_end = 3520
chunk_size = 50  # in MHz

# Initialize a list to store the aggregate results
aggregate_results = []

# Iterate over the frequency band in chunks of 25 MHz
for start in range(band_start, band_end, chunk_size):
    end = start + chunk_size
    # Filter the DataFrame for the current chunk
    ## 3350-3400
    #chunk_df = combined_df[(combined_df['frequency'] >= start) & (combined_df['frequency'] < end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385))]

    ## 3470-3520
    chunk_df = combined_df[(combined_df['frequency'] >= start) & (combined_df['frequency'] <= end)& ((combined_df['frequency'] <= 3433.5) | (combined_df['frequency'] >= 3434.8)) & ((combined_df['frequency'] <= 3458.5) | (combined_df['frequency'] >= 3459.8))]

    # Calculate mean and standard deviation in the linear domain
    mean_power_db = np.mean(chunk_df['power'])
    std_power_db = np.std(chunk_df['power'])

    # Set the threshold to mean + 1 standard deviation
    threshold = -145

    # Calculate the duty cycle and average power when occupied
    occupied_df = chunk_df[chunk_df['power'] > threshold]
    duty_cycle = len(occupied_df) / len(chunk_df) * 100
    avg_power_occupied = np.mean(occupied_df['power']) if not occupied_df.empty else np.nan

    # Store the results for the current chunk
    aggregate_results.append({
        'Chunk_Start': start,
        'Chunk_End': end,
        'Mean': mean_power_db,
        'Std': std_power_db,
        'Threshold': threshold,
        'Duty_Cycle': duty_cycle,
        'Avg_Power_Occupied': avg_power_occupied
    })

# Convert the list of dicts to a DataFrame
aggregate_results_df = pd.DataFrame(aggregate_results)

# Aggregate the results
# You can choose how to aggregate, here's a simple mean aggregation
final_mean_threshold = aggregate_results_df['Threshold'].mean()
final_duty_cycle = aggregate_results_df['Duty_Cycle'].mean()
final_avg_power_occupied = aggregate_results_df['Avg_Power_Occupied'].mean()

print(aggregate_results_df)
print(f"Final Mean Threshold: {final_mean_threshold}")
print(f"Final Duty Cycle: {final_duty_cycle}")
print(f"Final Average Power Occupied: {final_avg_power_occupied}")


# Filter rows where frequency is between band start and ends

## 3350-3400
#filtered_df = combined_df[(combined_df['frequency'] >= band_start) & (combined_df['frequency'] <= band_end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385.5))]

## 3470-3520
filtered_df = combined_df[(combined_df['frequency'] >= band_start) & (combined_df['frequency'] <= band_end)& ((combined_df['frequency'] <= 3433.5) | (combined_df['frequency'] >= 3434.8)) & ((combined_df['frequency'] <= 3458.5) | (combined_df['frequency'] >= 3459.8))]

plt.figure(figsize=(20, 15))
plt.scatter(filtered_df["frequency"], filtered_df["power"], marker="*", s=0.3)
plt.xlabel("Frequency (MHz)")
plt.ylabel("Power (dB)")
plt.title("Monitor @"+monitor_foldername)
plt.show()
# Define a power threshold, for example, -90 dB
power_threshold = threshold

# Count the number of rows above the power threshold
occupied_count = filtered_df[filtered_df['power'] > power_threshold].shape[0]

# Calculate the total number of rows in the filtered DataFrame
total_count = filtered_df.shape[0]

# Calculate the duty cycle
duty_cycle = (occupied_count / total_count) * 100

print(duty_cycle)

plt.figure()
plt.hist(filtered_df["power"], bins=20)
plt.show()

### Occupancy with respect to Time

In [ ]:

# Load your dataframe (df)

# Ensure that the 'timestamp' column is in datetime format
#combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'])

# Define the three intervals
intervals = {
    'morning': ('04:00:00', '12:00:00'),
    'afternoon': ('12:00:00', '20:00:00'),
    'night': ('20:00:00', '04:00:00')
}

# Divide the dataframe into multiple dataframes
dfs = {}
for period, (start, end) in intervals.items():
    if period == 'night':
        # This interval wraps around midnight, so we need to handle it differently
        mask = ((combined_df['timestamp'].dt.time >= pd.to_datetime(start).time()) |
                (combined_df['timestamp'].dt.time < pd.to_datetime(end).time()))
    else:
        mask = ((combined_df['timestamp'].dt.time >= pd.to_datetime(start).time()) & 
                (combined_df['timestamp'].dt.time < pd.to_datetime(end).time()))
    dfs[period] = combined_df.loc[mask]

# Now dfs is a dictionary with keys as the period names and values as the dataframes for those times


In [ ]:
(dfs['morning_afternoon']).head()

### Afternoon-Night

In [ ]:
intervals = {
    'morning': ('04:00:00', '12:00:00'),
    'afternoon': ('12:00:00', '20:00:00'),
    'night': ('20:00:00', '04:00:00')
}

# Divide the dataframe into multiple dataframes
dfs = {}
for period, (start, end) in intervals.items():
    if period == 'night':
        # This interval wraps around midnight, so we need to handle it differently
        mask = ((combined_df['timestamp'].dt.time >= pd.to_datetime(start).time()) |
                (combined_df['timestamp'].dt.time < pd.to_datetime(end).time()))
    else:
        mask = ((combined_df['timestamp'].dt.time >= pd.to_datetime(start).time()) & 
                (combined_df['timestamp'].dt.time < pd.to_datetime(end).time()))
    dfs[period] = combined_df.loc[mask]

# Assuming combined_df is your DataFrame
period = list(intervals.keys())[0]
combined_df_ma = dfs[period]
#df = combined_df.groupby('frequency')['power'].apply(avg_pow).reset_index()

# Define the start and end of the CBRS band (in MHz)
band_start = 3470
band_end = 3520
chunk_size = 50  # in MHz

# Initialize a list to store the aggregate results
aggregate_results = []

# Iterate over the frequency band in chunks of 25 MHz
for start in range(band_start, band_end, chunk_size):
    end = start + chunk_size
    # Filter the DataFrame for the current chunk
    ## 3350-3400
    #chunk_df = combined_df[(combined_df['frequency'] >= start) & (combined_df['frequency'] < end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385))]

    ## 3470-3520
    chunk_df = combined_df_ma[(combined_df_ma['frequency'] >= start) & (combined_df_ma['frequency'] <= end)& ((combined_df_ma['frequency'] <= 3433.5) | (combined_df_ma['frequency'] >= 3434.8)) & ((combined_df_ma['frequency'] <= 3458.5) | (combined_df_ma['frequency'] >= 3459.8))]

    # Calculate mean and standard deviation in the linear domain
    mean_power_db = np.mean(chunk_df['power'])
    std_power_db = np.std(chunk_df['power'])

    # Set the threshold to mean + 1 standard deviation
    threshold = -145

    # Calculate the duty cycle and average power when occupied
    occupied_df = chunk_df[chunk_df['power'] > threshold]
    duty_cycle = len(occupied_df) / len(chunk_df) * 100
    avg_power_occupied = np.mean(occupied_df['power']) if not occupied_df.empty else np.nan

    # Store the results for the current chunk
    aggregate_results.append({
        'Chunk_Start': start,
        'Chunk_End': end,
        'Mean': mean_power_db,
        'Std': std_power_db,
        'Threshold': threshold,
        'Duty_Cycle': duty_cycle,
        'Avg_Power_Occupied': avg_power_occupied
    })

# Convert the list of dicts to a DataFrame
aggregate_results_df = pd.DataFrame(aggregate_results)

# Aggregate the results
# You can choose how to aggregate, here's a simple mean aggregation
final_mean_threshold = aggregate_results_df['Threshold'].mean()
final_duty_cycle = aggregate_results_df['Duty_Cycle'].mean()
final_avg_power_occupied = aggregate_results_df['Avg_Power_Occupied'].mean()

print(aggregate_results_df)
print(f"Final Mean Threshold: {final_mean_threshold}")
print(f"Final Duty Cycle: {final_duty_cycle}")
print(f"Final Average Power Occupied: {final_avg_power_occupied}")


# Filter rows where frequency is between band start and ends

## 3350-3400
#filtered_df = combined_df[(combined_df['frequency'] >= band_start) & (combined_df['frequency'] <= band_end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385.5))]

## 3470-3520
filtered_df = combined_df_ma[(combined_df_ma['frequency'] >= band_start) & (combined_df_ma['frequency'] <= band_end)& ((combined_df_ma['frequency'] <= 3433.5) | (combined_df_ma['frequency'] >= 3434.8)) & ((combined_df_ma['frequency'] <= 3458.5) | (combined_df_ma['frequency'] >= 3459.8))]

plt.figure(figsize=(20, 15))
plt.scatter(filtered_df["frequency"], filtered_df["power"], marker="*", s=0.3)
plt.xlabel("Frequency (MHz)")
plt.ylabel("Power (dB)")
plt.title("Monitor @"+monitor_foldername+" Period "+period)
plt.show()
# Define a power threshold, for example, -90 dB
power_threshold = threshold

# Count the number of rows above the power threshold
occupied_count = filtered_df[filtered_df['power'] > power_threshold].shape[0]

# Calculate the total number of rows in the filtered DataFrame
total_count = filtered_df.shape[0]

# Calculate the duty cycle
duty_cycle = (occupied_count / total_count) * 100

print(duty_cycle)

plt.figure()
plt.hist(filtered_df["power"], bins=20)
plt.show()

### Night-Afternoon

In [ ]:

# Assuming combined_df is your DataFrame
period = list(intervals.keys())[1]
combined_df_ma = dfs[period]
#df = combined_df.groupby('frequency')['power'].apply(avg_pow).reset_index()

# Define the start and end of the CBRS band (in MHz)
band_start = 3470
band_end = 3520
chunk_size = 50  # in MHz

# Initialize a list to store the aggregate results
aggregate_results = []

# Iterate over the frequency band in chunks of 25 MHz
for start in range(band_start, band_end, chunk_size):
    end = start + chunk_size
    # Filter the DataFrame for the current chunk
    ## 3350-3400
    #chunk_df = combined_df[(combined_df['frequency'] >= start) & (combined_df['frequency'] < end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385))]

    ## 3470-3520
    chunk_df = combined_df_ma[(combined_df_ma['frequency'] >= start) & (combined_df_ma['frequency'] <= end)& ((combined_df_ma['frequency'] <= 3433.5) | (combined_df_ma['frequency'] >= 3434.8)) & ((combined_df_ma['frequency'] <= 3458.5) | (combined_df_ma['frequency'] >= 3459.8))]

    # Calculate mean and standard deviation in the linear domain
    mean_power_db = np.mean(chunk_df['power'])
    std_power_db = np.std(chunk_df['power'])

    # Set the threshold to mean + 1 standard deviation
    threshold = -145

    # Calculate the duty cycle and average power when occupied
    occupied_df = chunk_df[chunk_df['power'] > threshold]
    duty_cycle = len(occupied_df) / len(chunk_df) * 100
    avg_power_occupied = np.mean(occupied_df['power']) if not occupied_df.empty else np.nan

    # Store the results for the current chunk
    aggregate_results.append({
        'Chunk_Start': start,
        'Chunk_End': end,
        'Mean': mean_power_db,
        'Std': std_power_db,
        'Threshold': threshold,
        'Duty_Cycle': duty_cycle,
        'Avg_Power_Occupied': avg_power_occupied
    })

# Convert the list of dicts to a DataFrame
aggregate_results_df = pd.DataFrame(aggregate_results)

# Aggregate the results
# You can choose how to aggregate, here's a simple mean aggregation
final_mean_threshold = aggregate_results_df['Threshold'].mean()
final_duty_cycle = aggregate_results_df['Duty_Cycle'].mean()
final_avg_power_occupied = aggregate_results_df['Avg_Power_Occupied'].mean()

print(aggregate_results_df)
print(f"Final Mean Threshold: {final_mean_threshold}")
print(f"Final Duty Cycle: {final_duty_cycle}")
print(f"Final Average Power Occupied: {final_avg_power_occupied}")


# Filter rows where frequency is between band start and ends

## 3350-3400
#filtered_df = combined_df[(combined_df['frequency'] >= band_start) & (combined_df['frequency'] <= band_end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385.5))]

## 3470-3520
filtered_df = combined_df_ma[(combined_df_ma['frequency'] >= band_start) & (combined_df_ma['frequency'] <= band_end)& ((combined_df_ma['frequency'] <= 3433.5) | (combined_df_ma['frequency'] >= 3434.8)) & ((combined_df_ma['frequency'] <= 3458.5) | (combined_df_ma['frequency'] >= 3459.8))]

plt.figure(figsize=(20, 15))
plt.scatter(filtered_df["frequency"], filtered_df["power"], marker="*", s=0.3)
plt.xlabel("Frequency (MHz)")
plt.ylabel("Power (dB)")
plt.title("Monitor @"+monitor_foldername+" Period "+period)
plt.show()
# Define a power threshold, for example, -90 dB
power_threshold = threshold

# Count the number of rows above the power threshold
occupied_count = filtered_df[filtered_df['power'] > power_threshold].shape[0]

# Calculate the total number of rows in the filtered DataFrame
total_count = filtered_df.shape[0]

# Calculate the duty cycle
duty_cycle = (occupied_count / total_count) * 100

print(duty_cycle)

plt.figure()
plt.hist(filtered_df["power"], bins=20)
plt.show()

### Night

In [ ]:

# Assuming combined_df is your DataFrame
period = list(intervals.keys())[2]
combined_df_ma = dfs[period]
#df = combined_df.groupby('frequency')['power'].apply(avg_pow).reset_index()

# Define the start and end of the CBRS band (in MHz)
band_start = 3470
band_end = 3520
chunk_size = 50  # in MHz

# Initialize a list to store the aggregate results
aggregate_results = []

# Iterate over the frequency band in chunks of 25 MHz
for start in range(band_start, band_end, chunk_size):
    end = start + chunk_size
    # Filter the DataFrame for the current chunk
    ## 3350-3400
    #chunk_df = combined_df[(combined_df['frequency'] >= start) & (combined_df['frequency'] < end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385))]

    ## 3470-3520
    chunk_df = combined_df_ma[(combined_df_ma['frequency'] >= start) & (combined_df_ma['frequency'] <= end)& ((combined_df_ma['frequency'] <= 3433.5) | (combined_df_ma['frequency'] >= 3434.8)) & ((combined_df_ma['frequency'] <= 3458.5) | (combined_df_ma['frequency'] >= 3459.8))]

    # Calculate mean and standard deviation in the linear domain
    mean_power_db = np.mean(chunk_df['power'])
    std_power_db = np.std(chunk_df['power'])

    # Set the threshold to mean + 1 standard deviation
    threshold = -145

    # Calculate the duty cycle and average power when occupied
    occupied_df = chunk_df[chunk_df['power'] > threshold]
    duty_cycle = len(occupied_df) / len(chunk_df) * 100
    avg_power_occupied = np.mean(occupied_df['power']) if not occupied_df.empty else np.nan

    # Store the results for the current chunk
    aggregate_results.append({
        'Chunk_Start': start,
        'Chunk_End': end,
        'Mean': mean_power_db,
        'Std': std_power_db,
        'Threshold': threshold,
        'Duty_Cycle': duty_cycle,
        'Avg_Power_Occupied': avg_power_occupied
    })

# Convert the list of dicts to a DataFrame
aggregate_results_df = pd.DataFrame(aggregate_results)

# Aggregate the results
# You can choose how to aggregate, here's a simple mean aggregation
final_mean_threshold = aggregate_results_df['Threshold'].mean()
final_duty_cycle = aggregate_results_df['Duty_Cycle'].mean()
final_avg_power_occupied = aggregate_results_df['Avg_Power_Occupied'].mean()

print(aggregate_results_df)
print(f"Final Mean Threshold: {final_mean_threshold}")
print(f"Final Duty Cycle: {final_duty_cycle}")
print(f"Final Average Power Occupied: {final_avg_power_occupied}")


# Filter rows where frequency is between band start and ends

## 3350-3400
#filtered_df = combined_df[(combined_df['frequency'] >= band_start) & (combined_df['frequency'] <= band_end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385.5))]

## 3470-3520
filtered_df = combined_df_ma[(combined_df_ma['frequency'] >= band_start) & (combined_df_ma['frequency'] <= band_end)& ((combined_df_ma['frequency'] <= 3433.5) | (combined_df_ma['frequency'] >= 3434.8)) & ((combined_df_ma['frequency'] <= 3458.5) | (combined_df_ma['frequency'] >= 3459.8))]

plt.figure(figsize=(20, 15))
plt.scatter(filtered_df["frequency"], filtered_df["power"], marker="*", s=0.3)
plt.xlabel("Frequency (MHz)")
plt.ylabel("Power (dB)")
plt.title("Monitor @"+monitor_foldername+" Period "+period)
plt.show()
# Define a power threshold, for example, -90 dB
power_threshold = threshold

# Count the number of rows above the power threshold
occupied_count = filtered_df[filtered_df['power'] > power_threshold].shape[0]

# Calculate the total number of rows in the filtered DataFrame
total_count = filtered_df.shape[0]

# Calculate the duty cycle
duty_cycle = (occupied_count / total_count) * 100

print(duty_cycle)

plt.figure()
plt.hist(filtered_df["power"], bins=20)
plt.show()

### Divide According to Season

In [ ]:
# Load your dataframe (df)

# Ensure that the 'timestamp' column is in datetime format
#combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'])

# Define the three intervals
seasons = {
    'spring': (3, 6),
    'summer': (6, 9),
    'autumn': (9, 12),
    'winter': (0, 3)
}

# Divide the dataframe into multiple dataframes
dfs = {}
for season, (start, end) in seasons.items():
    mask = ((combined_df['timestamp'].dt.month >= start) &
            (combined_df['timestamp'].dt.month <= end))

    dfs[season] = combined_df.loc[mask]

# Now dfs is a dictionary with keys as the period names and values as the dataframes for those times


### Spring

In [ ]:

# Assuming combined_df is your DataFrame
period = list(seasons.keys())[0]
combined_df_ma = dfs[period]
#df = combined_df.groupby('frequency')['power'].apply(avg_pow).reset_index()

# Define the start and end of the CBRS band (in MHz)
band_start = 3470
band_end = 3520
chunk_size = 50  # in MHz

# Initialize a list to store the aggregate results
aggregate_results = []

# Iterate over the frequency band in chunks of 25 MHz
for start in range(band_start, band_end, chunk_size):
    end = start + chunk_size
    # Filter the DataFrame for the current chunk
    ## 3350-3400
    #chunk_df = combined_df[(combined_df['frequency'] >= start) & (combined_df['frequency'] < end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385))]

    ## 3470-3520
    chunk_df = combined_df_ma[(combined_df_ma['frequency'] >= start) & (combined_df_ma['frequency'] <= end)& ((combined_df_ma['frequency'] <= 3433.5) | (combined_df_ma['frequency'] >= 3434.8)) & ((combined_df_ma['frequency'] <= 3458.5) | (combined_df_ma['frequency'] >= 3459.8))]

    # Calculate mean and standard deviation in the linear domain
    mean_power_db = np.mean(chunk_df['power'])
    std_power_db = np.std(chunk_df['power'])

    # Set the threshold to mean + 1 standard deviation
    threshold = -145

    # Calculate the duty cycle and average power when occupied
    occupied_df = chunk_df[chunk_df['power'] > threshold]
    duty_cycle = len(occupied_df) / len(chunk_df) * 100
    avg_power_occupied = np.mean(occupied_df['power']) if not occupied_df.empty else np.nan

    # Store the results for the current chunk
    aggregate_results.append({
        'Chunk_Start': start,
        'Chunk_End': end,
        'Mean': mean_power_db,
        'Std': std_power_db,
        'Threshold': threshold,
        'Duty_Cycle': duty_cycle,
        'Avg_Power_Occupied': avg_power_occupied
    })

# Convert the list of dicts to a DataFrame
aggregate_results_df = pd.DataFrame(aggregate_results)

# Aggregate the results
# You can choose how to aggregate, here's a simple mean aggregation
final_mean_threshold = aggregate_results_df['Threshold'].mean()
final_duty_cycle = aggregate_results_df['Duty_Cycle'].mean()
final_avg_power_occupied = aggregate_results_df['Avg_Power_Occupied'].mean()

print(aggregate_results_df)
print(f"Final Mean Threshold: {final_mean_threshold}")
print(f"Final Duty Cycle: {final_duty_cycle}")
print(f"Final Average Power Occupied: {final_avg_power_occupied}")


# Filter rows where frequency is between band start and ends

## 3350-3400
#filtered_df = combined_df[(combined_df['frequency'] >= band_start) & (combined_df['frequency'] <= band_end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385.5))]

## 3470-3520
filtered_df = combined_df_ma[(combined_df_ma['frequency'] >= band_start) & (combined_df_ma['frequency'] <= band_end)& ((combined_df_ma['frequency'] <= 3433.5) | (combined_df_ma['frequency'] >= 3434.8)) & ((combined_df_ma['frequency'] <= 3458.5) | (combined_df_ma['frequency'] >= 3459.8))]

plt.figure(figsize=(20, 15))
plt.scatter(filtered_df["frequency"], filtered_df["power"], marker="*", s=0.3)
plt.xlabel("Frequency (MHz)")
plt.ylabel("Power (dB)")
plt.title("Monitor @"+monitor_foldername+" Period "+period)
plt.show()
# Define a power threshold, for example, -90 dB
power_threshold = threshold

# Count the number of rows above the power threshold
occupied_count = filtered_df[filtered_df['power'] > power_threshold].shape[0]

# Calculate the total number of rows in the filtered DataFrame
total_count = filtered_df.shape[0]

# Calculate the duty cycle
duty_cycle = (occupied_count / total_count) * 100

print(duty_cycle)

plt.figure()
plt.hist(filtered_df["power"], bins=20)
plt.show()

### Summer

In [ ]:

# Assuming combined_df is your DataFrame
period = list(seasons.keys())[1]
combined_df_ma = dfs[period]
#df = combined_df.groupby('frequency')['power'].apply(avg_pow).reset_index()

# Define the start and end of the CBRS band (in MHz)
band_start = 3470
band_end = 3520
chunk_size = 50  # in MHz

# Initialize a list to store the aggregate results
aggregate_results = []

# Iterate over the frequency band in chunks of 25 MHz
for start in range(band_start, band_end, chunk_size):
    end = start + chunk_size
    # Filter the DataFrame for the current chunk
    ## 3350-3400
    #chunk_df = combined_df[(combined_df['frequency'] >= start) & (combined_df['frequency'] < end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385))]

    ## 3470-3520
    chunk_df = combined_df_ma[(combined_df_ma['frequency'] >= start) & (combined_df_ma['frequency'] <= end)& ((combined_df_ma['frequency'] <= 3433.5) | (combined_df_ma['frequency'] >= 3434.8)) & ((combined_df_ma['frequency'] <= 3458.5) | (combined_df_ma['frequency'] >= 3459.8))]

    # Calculate mean and standard deviation in the linear domain
    mean_power_db = np.mean(chunk_df['power'])
    std_power_db = np.std(chunk_df['power'])

    # Set the threshold to mean + 1 standard deviation
    threshold = -145

    # Calculate the duty cycle and average power when occupied
    occupied_df = chunk_df[chunk_df['power'] > threshold]
    duty_cycle = len(occupied_df) / len(chunk_df) * 100
    avg_power_occupied = np.mean(occupied_df['power']) if not occupied_df.empty else np.nan

    # Store the results for the current chunk
    aggregate_results.append({
        'Chunk_Start': start,
        'Chunk_End': end,
        'Mean': mean_power_db,
        'Std': std_power_db,
        'Threshold': threshold,
        'Duty_Cycle': duty_cycle,
        'Avg_Power_Occupied': avg_power_occupied
    })

# Convert the list of dicts to a DataFrame
aggregate_results_df = pd.DataFrame(aggregate_results)

# Aggregate the results
# You can choose how to aggregate, here's a simple mean aggregation
final_mean_threshold = aggregate_results_df['Threshold'].mean()
final_duty_cycle = aggregate_results_df['Duty_Cycle'].mean()
final_avg_power_occupied = aggregate_results_df['Avg_Power_Occupied'].mean()

print(aggregate_results_df)
print(f"Final Mean Threshold: {final_mean_threshold}")
print(f"Final Duty Cycle: {final_duty_cycle}")
print(f"Final Average Power Occupied: {final_avg_power_occupied}")


# Filter rows where frequency is between band start and ends

## 3350-3400
#filtered_df = combined_df[(combined_df['frequency'] >= band_start) & (combined_df['frequency'] <= band_end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385.5))]

## 3470-3520
filtered_df = combined_df_ma[(combined_df_ma['frequency'] >= band_start) & (combined_df_ma['frequency'] <= band_end)& ((combined_df_ma['frequency'] <= 3433.5) | (combined_df_ma['frequency'] >= 3434.8)) & ((combined_df_ma['frequency'] <= 3458.5) | (combined_df_ma['frequency'] >= 3459.8))]

plt.figure(figsize=(20, 15))
plt.scatter(filtered_df["frequency"], filtered_df["power"], marker="*", s=0.3)
plt.xlabel("Frequency (MHz)")
plt.ylabel("Power (dB)")
plt.title("Monitor @"+monitor_foldername+" Period "+period)
plt.show()
# Define a power threshold, for example, -90 dB
power_threshold = threshold

# Count the number of rows above the power threshold
occupied_count = filtered_df[filtered_df['power'] > power_threshold].shape[0]

# Calculate the total number of rows in the filtered DataFrame
total_count = filtered_df.shape[0]

# Calculate the duty cycle
duty_cycle = (occupied_count / total_count) * 100

print(duty_cycle)

plt.figure()
plt.hist(filtered_df["power"], bins=20)
plt.show()

### Fall

In [ ]:

# Assuming combined_df is your DataFrame
period = list(seasons.keys())[2]
combined_df_ma = dfs[period]
#df = combined_df.groupby('frequency')['power'].apply(avg_pow).reset_index()

# Define the start and end of the CBRS band (in MHz)
band_start = 3470
band_end = 3520
chunk_size = 50  # in MHz

# Initialize a list to store the aggregate results
aggregate_results = []

# Iterate over the frequency band in chunks of 25 MHz
for start in range(band_start, band_end, chunk_size):
    end = start + chunk_size
    # Filter the DataFrame for the current chunk
    ## 3350-3400
    #chunk_df = combined_df[(combined_df['frequency'] >= start) & (combined_df['frequency'] < end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385))]

    ## 3470-3520
    chunk_df = combined_df_ma[(combined_df_ma['frequency'] >= start) & (combined_df_ma['frequency'] <= end)& ((combined_df_ma['frequency'] <= 3433.5) | (combined_df_ma['frequency'] >= 3434.8)) & ((combined_df_ma['frequency'] <= 3458.5) | (combined_df_ma['frequency'] >= 3459.8))]

    # Calculate mean and standard deviation in the linear domain
    mean_power_db = np.mean(chunk_df['power'])
    std_power_db = np.std(chunk_df['power'])

    # Set the threshold to mean + 1 standard deviation
    threshold = -145

    # Calculate the duty cycle and average power when occupied
    occupied_df = chunk_df[chunk_df['power'] > threshold]
    duty_cycle = len(occupied_df) / len(chunk_df) * 100
    avg_power_occupied = np.mean(occupied_df['power']) if not occupied_df.empty else np.nan

    # Store the results for the current chunk
    aggregate_results.append({
        'Chunk_Start': start,
        'Chunk_End': end,
        'Mean': mean_power_db,
        'Std': std_power_db,
        'Threshold': threshold,
        'Duty_Cycle': duty_cycle,
        'Avg_Power_Occupied': avg_power_occupied
    })

# Convert the list of dicts to a DataFrame
aggregate_results_df = pd.DataFrame(aggregate_results)

# Aggregate the results
# You can choose how to aggregate, here's a simple mean aggregation
final_mean_threshold = aggregate_results_df['Threshold'].mean()
final_duty_cycle = aggregate_results_df['Duty_Cycle'].mean()
final_avg_power_occupied = aggregate_results_df['Avg_Power_Occupied'].mean()

print(aggregate_results_df)
print(f"Final Mean Threshold: {final_mean_threshold}")
print(f"Final Duty Cycle: {final_duty_cycle}")
print(f"Final Average Power Occupied: {final_avg_power_occupied}")


# Filter rows where frequency is between band start and ends

## 3350-3400
#filtered_df = combined_df[(combined_df['frequency'] >= band_start) & (combined_df['frequency'] <= band_end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385.5))]

## 3470-3520
filtered_df = combined_df_ma[(combined_df_ma['frequency'] >= band_start) & (combined_df_ma['frequency'] <= band_end)& ((combined_df_ma['frequency'] <= 3433.5) | (combined_df_ma['frequency'] >= 3434.8)) & ((combined_df_ma['frequency'] <= 3458.5) | (combined_df_ma['frequency'] >= 3459.8))]

plt.figure(figsize=(20, 15))
plt.scatter(filtered_df["frequency"], filtered_df["power"], marker="*", s=0.3)
plt.xlabel("Frequency (MHz)")
plt.ylabel("Power (dB)")
plt.title("Monitor @"+monitor_foldername+" Period "+period)
plt.show()
# Define a power threshold, for example, -90 dB
power_threshold = threshold

# Count the number of rows above the power threshold
occupied_count = filtered_df[filtered_df['power'] > power_threshold].shape[0]

# Calculate the total number of rows in the filtered DataFrame
total_count = filtered_df.shape[0]

# Calculate the duty cycle
duty_cycle = (occupied_count / total_count) * 100

print(duty_cycle)

plt.figure()
plt.hist(filtered_df["power"], bins=20)
plt.show()

### Winter

In [ ]:

# Assuming combined_df is your DataFrame
period = list(seasons.keys())[3]
combined_df_ma = dfs[period]
#df = combined_df.groupby('frequency')['power'].apply(avg_pow).reset_index()

# Define the start and end of the CBRS band (in MHz)
band_start = 3470
band_end = 3520
chunk_size = 50  # in MHz

# Initialize a list to store the aggregate results
aggregate_results = []

# Iterate over the frequency band in chunks of 25 MHz
for start in range(band_start, band_end, chunk_size):
    end = start + chunk_size
    # Filter the DataFrame for the current chunk
    ## 3350-3400
    #chunk_df = combined_df[(combined_df['frequency'] >= start) & (combined_df['frequency'] < end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385))]

    ## 3470-3520
    chunk_df = combined_df_ma[(combined_df_ma['frequency'] >= start) & (combined_df_ma['frequency'] <= end)& ((combined_df_ma['frequency'] <= 3433.5) | (combined_df_ma['frequency'] >= 3434.8)) & ((combined_df_ma['frequency'] <= 3458.5) | (combined_df_ma['frequency'] >= 3459.8))]

    # Calculate mean and standard deviation in the linear domain
    mean_power_db = np.mean(chunk_df['power'])
    std_power_db = np.std(chunk_df['power'])

    # Set the threshold to mean + 1 standard deviation
    threshold = -145

    # Calculate the duty cycle and average power when occupied
    occupied_df = chunk_df[chunk_df['power'] > threshold]
    duty_cycle = len(occupied_df) / len(chunk_df) * 100
    avg_power_occupied = np.mean(occupied_df['power']) if not occupied_df.empty else np.nan

    # Store the results for the current chunk
    aggregate_results.append({
        'Chunk_Start': start,
        'Chunk_End': end,
        'Mean': mean_power_db,
        'Std': std_power_db,
        'Threshold': threshold,
        'Duty_Cycle': duty_cycle,
        'Avg_Power_Occupied': avg_power_occupied
    })

# Convert the list of dicts to a DataFrame
aggregate_results_df = pd.DataFrame(aggregate_results)

# Aggregate the results
# You can choose how to aggregate, here's a simple mean aggregation
final_mean_threshold = aggregate_results_df['Threshold'].mean()
final_duty_cycle = aggregate_results_df['Duty_Cycle'].mean()
final_avg_power_occupied = aggregate_results_df['Avg_Power_Occupied'].mean()

print(aggregate_results_df)
print(f"Final Mean Threshold: {final_mean_threshold}")
print(f"Final Duty Cycle: {final_duty_cycle}")
print(f"Final Average Power Occupied: {final_avg_power_occupied}")


# Filter rows where frequency is between band start and ends

## 3350-3400
#filtered_df = combined_df[(combined_df['frequency'] >= band_start) & (combined_df['frequency'] <= band_end)& ((combined_df['frequency'] <= 3358.5) | (combined_df['frequency'] >= 3361)) & ((combined_df['frequency'] <= 3383) | (combined_df['frequency'] >= 3385.5))]

## 3470-3520
filtered_df = combined_df_ma[(combined_df_ma['frequency'] >= band_start) & (combined_df_ma['frequency'] <= band_end)& ((combined_df_ma['frequency'] <= 3433.5) | (combined_df_ma['frequency'] >= 3434.8)) & ((combined_df_ma['frequency'] <= 3458.5) | (combined_df_ma['frequency'] >= 3459.8))]

plt.figure(figsize=(20, 15))
plt.scatter(filtered_df["frequency"], filtered_df["power"], marker="*", s=0.3)
plt.xlabel("Frequency (MHz)")
plt.ylabel("Power (dB)")
plt.title("Monitor @"+monitor_foldername+" Period "+period)
plt.show()
# Define a power threshold, for example, -90 dB
power_threshold = threshold

# Count the number of rows above the power threshold
occupied_count = filtered_df[filtered_df['power'] > power_threshold].shape[0]

# Calculate the total number of rows in the filtered DataFrame
total_count = filtered_df.shape[0]

# Calculate the duty cycle
duty_cycle = (occupied_count / total_count) * 100

print(duty_cycle)

plt.figure()
plt.hist(filtered_df["power"], bins=20)
plt.show()